In [6]:
#!pip install tedana
%matplotlib inline
import os
import os.path as op
from glob import glob
import webbrowser

from tedana import (combine, decay, io, utils, __version__)
from tedana.workflows import tedana_workflow
from tedana.workflows import t2smap_workflow
from tedana.workflows.parser_utils import is_valid_file
from tedana.due import due, BibTeX
#from tedana._utils import tedana_utils

import numpy as np
import nibabel as nib
from scipy import ndimage
from scipy import stats
from nilearn._utils import check_niimg
from sklearn.utils import check_array

from threadpoolctl import threadpool_limits
import argparse

In [2]:
%%time
#files = sorted(glob(op.join(dset_dir5, 'p06*.nii.gz')))
basedir =  '/Users/suhwan/Dropbox/Projects/sync_SEP/data/coil_test/20210803_multi_echo/preprocessed/1_2_3ME_HALF_CMRR_2_7ISO_PA_64CH+FLEX_MB4_IA2_0007/'
outdir =  '/Users/suhwan/Dropbox/Projects/sync_SEP/data/coil_test/20210803_multi_echo/preprocessed/1_2_3ME_HALF_CMRR_2_7ISO_PA_64CH+FLEX_MB4_IA2_0007/tedana_outputs'
outdir2 =  '/Users/suhwan/Dropbox/Projects/sync_SEP/data/coil_test/20210803_multi_echo/preprocessed/1_2_3ME_HALF_CMRR_2_7ISO_PA_64CH+FLEX_MB4_IA2_0007/t2smap_outputs'
files2 = sorted(glob(op.join(basedir, 'ra*e*.nii')))
files2 

CPU times: user 749 µs, sys: 914 µs, total: 1.66 ms
Wall time: 850 µs


['/Users/suhwan/Dropbox/Projects/sync_SEP/data/coil_test/20210803_multi_echo/preprocessed/1_2_3ME_HALF_CMRR_2_7ISO_PA_64CH+FLEX_MB4_IA2_0007/ra20210803111531_1_2_3ME_half_CMRR_2.7iso_PA_64ch+Flex_mb4_IA2_7_e1.nii',
 '/Users/suhwan/Dropbox/Projects/sync_SEP/data/coil_test/20210803_multi_echo/preprocessed/1_2_3ME_HALF_CMRR_2_7ISO_PA_64CH+FLEX_MB4_IA2_0007/ra20210803111531_1_2_3ME_half_CMRR_2.7iso_PA_64ch+Flex_mb4_IA2_7_e2.nii',
 '/Users/suhwan/Dropbox/Projects/sync_SEP/data/coil_test/20210803_multi_echo/preprocessed/1_2_3ME_HALF_CMRR_2_7ISO_PA_64CH+FLEX_MB4_IA2_0007/ra20210803111531_1_2_3ME_half_CMRR_2.7iso_PA_64ch+Flex_mb4_IA2_7_e3.nii']

In [8]:
catd, ref_img = io.load_data(files2, n_echos=3)
getsum=True
threshold=1
mask = 'none'

In [9]:
# take temporal mean of echos and extract non-zero values in first echo
echo_means = catd.mean(axis=-1)  # temporal mean of echos
first_echo = echo_means[echo_means[:, 0] != 0, 0]

# get 33rd %ile of `first_echo` and find corresponding index
# NOTE: percentile is arbitrary
perc = np.percentile(first_echo, 33, interpolation='higher')
perc_val = (echo_means[:, 0] == perc)

(3,)

In [13]:
# extract values from all echos at relevant index
# NOTE: threshold of 1/3 voxel value is arbitrary
lthrs = np.squeeze(echo_means[perc_val].T) / 3

# if multiple samples were extracted per echo, keep the one w/the highest signal
if lthrs.ndim > 1:
    lthrs = lthrs[:, lthrs.sum(axis=0).argmax()]

In [17]:
# determine samples where absolute value is greater than echo-specific thresholds
# and count # of echos that pass criterion
masksum = (np.abs(echo_means) > lthrs).sum(axis=-1)
masksum

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# make it a boolean mask to (where we have at least `threshold` echoes with good signal)
mask = (masksum >= threshold).astype(bool)
masksum[masksum < threshold] = 0

echo_means = data.mean(axis=-1)  # temporal mean of echos
first_echo = echo_means[echo_means[:, 0] != 0, 0]